In [10]:
import gurobipy as gp

In [11]:
# Dados brutos
qtd_periodos = 12
estoque_inicial = 300
estoque_final = 300
custo_setup = 500
custo_estoque = 2
m_grande = 9999
vet_demandas = [185, 250, 0, 120, 150, 200, 0, 0, 180, 260, 0, 140]
vet_recursos = [30, 30, 30, 40, 40, 40, 40, 50, 50, 50, 50, 50]
recurso_unid = 0.2
qtd_periodos = len(vet_demandas)
print(qtd_periodos)

12


In [12]:
periodos = ["T{}".format(t+1) for t in range(qtd_periodos)]

print(periodos)
demandas = dict()
for t, per in enumerate(periodos):
    demandas[per] = vet_demandas[t]

# Dicionário de recursos
recursos = dict()
for t, per in enumerate(periodos):
    recursos[per] = vet_recursos[t]
    

['T1', 'T2', 'T3', 'T4', 'T5', 'T6', 'T7', 'T8', 'T9', 'T10', 'T11', 'T12']


In [13]:
#Modelo de Programação Linear
m=gp.Model()

#variaveis de decisão 
producao = m.addVars(periodos, vtype = gp.GRB.INTEGER)
estoque = m.addVars(periodos, vtype =gp.GRB.INTEGER )
setup = m.addVars (periodos, vtype = gp.GRB.BINARY)

#Função Objetivo
m.setObjective(gp.quicksum(setup[t]*custo_setup + estoque[t]*custo_estoque for t in periodos ), sense = gp.GRB.MINIMIZE)

#Restrição para o Periodo 1
m.addConstr(producao["T1"] + estoque_inicial - demandas["T1"] == estoque["T1"])

m.addConstrs(producao[t] + estoque[periodos[idx_t]] - demandas[t] == estoque[t] for idx_t, t in enumerate(periodos[1:]))

#Restrição que Garante a que o Total de Recursos por período não será excedido
m.addConstrs(recurso_unid * producao[t] <= recursos[t]*setup[t] for t in periodos)

m.addConstr(estoque[periodos[-1]]==estoque_final)

m.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 25 rows, 36 columns and 60 nonzeros
Model fingerprint: 0x468fa3be
Variable types: 0 continuous, 36 integer (12 binary)
Coefficient statistics:
  Matrix range     [2e-01, 5e+01]
  Objective range  [2e+00, 5e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+02, 3e+02]
Found heuristic solution: objective 5230.0000000
Presolve removed 8 rows and 8 columns
Presolve time: 0.00s
Presolved: 17 rows, 28 columns, 44 nonzeros
Variable types: 0 continuous, 28 integer (12 binary)

Root relaxation: objective 4.615000e+03, 16 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0      5230.00000 5230.00000  0.00%     -    0s

Explored 0 nodes (21 simplex iterations) in 0.07 seconds
Thread c

In [14]:
# Imprime a solução
for t in periodos:
    prod = round(producao[t].X)
    est = round(estoque[t].X)
    print("Produzir: {} unidades (estoque: {} unid)".format(prod, est))

Produzir: 0 unidades (estoque: 115 unid)
Produzir: 135 unidades (estoque: 0 unid)
Produzir: 0 unidades (estoque: 0 unid)
Produzir: 120 unidades (estoque: 0 unid)
Produzir: 150 unidades (estoque: 0 unid)
Produzir: 200 unidades (estoque: 0 unid)
Produzir: 0 unidades (estoque: 0 unid)
Produzir: 0 unidades (estoque: 0 unid)
Produzir: 190 unidades (estoque: 10 unid)
Produzir: 250 unidades (estoque: 0 unid)
Produzir: 190 unidades (estoque: 190 unid)
Produzir: 250 unidades (estoque: 300 unid)


In [15]:
#Imprime a Quantidade de Recursos utilizados

for t in periodos:
    usado = round (producao[t].X)*recurso_unid
    print("Periodo {} usou {} de recursos".format(t, usado))

Periodo T1 usou 0.0 de recursos
Periodo T2 usou 27.0 de recursos
Periodo T3 usou 0.0 de recursos
Periodo T4 usou 24.0 de recursos
Periodo T5 usou 30.0 de recursos
Periodo T6 usou 40.0 de recursos
Periodo T7 usou 0.0 de recursos
Periodo T8 usou 0.0 de recursos
Periodo T9 usou 38.0 de recursos
Periodo T10 usou 50.0 de recursos
Periodo T11 usou 38.0 de recursos
Periodo T12 usou 50.0 de recursos
